In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.common.exceptions import NoSuchElementException,TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


In [2]:
url = "https://sps.mesago.com/nuernberg/en/exhibitor-search.html"
if os.stat("all_files.html").st_size==0:
    chrome_driver=webdriver.Chrome()
    chrome_driver.maximize_window()
    chrome_driver.get(url)
    while True:
        chrome_driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        try:
            element_present=EC.presence_of_element_located((By.CSS_SELECTOR,".a-cubespinner.a-cubespinner--medium.center-block"))
            WebDriverWait(chrome_driver,10).until(element_present)
        except Exception as e:
            print(f"{e} aniticipated and handled")
            break
    with open("all_files.html","w") as file:
        root=BeautifulSoup(chrome_driver.page_source)
        all_hrefs=root.find_all("a",class_="a-link--no-focus")
        for each in all_hrefs:
            file.write(each["href"]+"\n")
else:
    print("HTML loaded")

HTML loaded


In [3]:
c_options=Options()
c_options.add_argument("--blink-settings=imagesEnabled=false")
new_driver=webdriver.Chrome(options=c_options)
wait=WebDriverWait(new_driver,86400)
new_driver.maximize_window()
lst=[]
url_lst=[]
count=0
with open("all_files.html","r") as file:
   links=file.readlines()
   for each in links:
      count+=1
      full_url="https://sps.mesago.com"+each
      new_driver.get(full_url)
      element_present=wait.until(EC.presence_of_element_located((By.CLASS_NAME,"ex-contact-box__flex")))
      new_driver.execute_script("return window.stop();")
      root_two=BeautifulSoup(new_driver.page_source,"html.parser")
      target_elem=root_two.find("div",class_="ex-contact-box__container")
      if target_elem:
         lst.append(target_elem)
         url_lst.append(full_url)
         print(count)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.177)
Stacktrace:
#0 0x55e1852bbd93 <unknown>
#1 0x55e18508a2d7 <unknown>
#2 0x55e185065655 <unknown>
#3 0x55e1850ef4ff <unknown>
#4 0x55e185102066 <unknown>
#5 0x55e1850ea353 <unknown>
#6 0x55e1850b9e40 <unknown>
#7 0x55e1850bb038 <unknown>
#8 0x55e18530f8be <unknown>
#9 0x55e1853138f0 <unknown>
#10 0x55e1852f3f90 <unknown>
#11 0x55e185314b7d <unknown>
#12 0x55e1852e5578 <unknown>
#13 0x55e185339348 <unknown>
#14 0x55e1853394d6 <unknown>
#15 0x55e185353341 <unknown>
#16 0x7f017a494b43 <unknown>


In [9]:
lst_two=[]
for t_url,each in zip(url_lst,lst):
    p_data=[]
    try:
        address=each.find("div",class_="ex-contact-box__container-address")
        links=each.find("div",class_="ex-contact-box__container-links")
        try:
            link=links.find("a",class_="icon icon-news-before ex-contact-box__website-link")["href"]
        except:
            link=None
        # addresses
        try:
            name=address.find("p",class_="ex-contact-box__address-field-name").text
            street=address.find("p",class_="ex-contact-box__address-field-street").text
            country=address.find("span",class_="ex-contact-box__address-field-country").text
            zip_=address.find("span",class_="ex-contact-box__address-field-zip").text
            city_=address.find("span",class_="ex-contact-box__address-field-city").text
        except:
            phone_=None
            street=None
            country=None
            zip_=None
            city_=None
        mail=address.find("a",class_="ex-contact-box__contact-btn btn btn-primary")["href"]
        p_data.extend([name,street,country,zip_,city_,phone_,mail,t_url,link])
        lst_two.append(p_data)
    except:
        continue
print(lst_two)

[]


In [117]:
import csv

rows=["Company name","Street address","Country","Zip","City","Phone Number","Email","URL","Web Address"]
with open("c_data_two.csv","w") as file:
    write=csv.writer(file)
    write.writerow(rows)
    write.writerows(lst_two)